In [40]:
import numpy as np
import pandas as pd
from tqdm import trange, tqdm
from sklearn.mixture import BayesianGaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.metrics import adjusted_rand_score
from joblib import dump, load
import time

# Data prep

In [41]:
# raw_data = pd.read_csv('../input/tabular-playground-series-jul-2022/data.csv')
raw_data = pd.read_csv('data\data.csv')
df = raw_data.drop(columns = 'id')

cols = [F'f_{i:02d}' for i in list(range(7,14))+list(range(22,29))]  # LGBM confirms that choice with feature_importance_
df = df[cols]

df_scaled = pd.DataFrame(PowerTransformer().fit_transform(df), columns=cols)
df_scaled.shape

(98000, 14)

# Sklego BayesianGMMClassifier

In [42]:
# SKLEGO BayesianGMMClassifier
# This a a copy of the BayesianGMMClassifier from SKLEGO to avoid the need to install the package
import numpy as np
from scipy.special import softmax
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.mixture import BayesianGaussianMixture
from sklearn.utils import check_X_y
from sklearn.utils.multiclass import unique_labels
from sklearn.utils.validation import check_is_fitted, check_array, FLOAT_DTYPES


class BayesianGMMClassifier(BaseEstimator, ClassifierMixin):
    def __init__(
        self,
        n_components=1,
        covariance_type="full",
        tol=0.001,
        reg_covar=1e-06,
        max_iter=100,
        n_init=1,
        init_params="kmeans",
        weight_concentration_prior_type="dirichlet_process",
        weight_concentration_prior=None,
        mean_precision_prior=None,
        mean_prior=None,
        degrees_of_freedom_prior=None,
        covariance_prior=None,
        random_state=None,
        warm_start=False,
        verbose=0,
        verbose_interval=10,
    ):
        """
        The BayesianGMMClassifier trains a Gaussian Mixture Model for each class in y on a dataset X. Once
        a density is trained for each class we can evaluate the likelihood scores to see which class
        is more likely. All parameters of the model are an exact copy of the parameters in scikit-learn.
        """
        self.n_components = n_components
        self.covariance_type = covariance_type
        self.tol = tol
        self.reg_covar = reg_covar
        self.max_iter = max_iter
        self.n_init = n_init
        self.init_params = init_params
        self.weight_concentration_prior_type = weight_concentration_prior_type
        self.weight_concentration_prior = weight_concentration_prior
        self.mean_precision_prior = mean_precision_prior
        self.mean_prior = mean_prior
        self.degrees_of_freedom_prior = degrees_of_freedom_prior
        self.covariance_prior = covariance_prior
        self.random_state = random_state
        self.warm_start = warm_start
        self.verbose = verbose
        self.verbose_interval = verbose_interval

    def fit(self, X: np.array, y: np.array) -> "BayesianGMMClassifier":
        """
        Fit the model using X, y as training data.

        :param X: array-like, shape=(n_columns, n_samples, ) training data.
        :param y: array-like, shape=(n_samples, ) training data.
        :return: Returns an instance of self.
        """
        X, y = check_X_y(X, y, estimator=self, dtype=FLOAT_DTYPES)
        if X.ndim == 1:
            X = np.expand_dims(X, 1)

        self.gmms_ = {}
        self.classes_ = unique_labels(y)
        for c in self.classes_:
            subset_x, subset_y = X[y == c], y[y == c]
            mixture = BayesianGaussianMixture(
                n_components=self.n_components,
                covariance_type=self.covariance_type,
                tol=self.tol,
                reg_covar=self.reg_covar,
                max_iter=self.max_iter,
                n_init=self.n_init,
                init_params=self.init_params,
                weight_concentration_prior_type=self.weight_concentration_prior_type,
                weight_concentration_prior=self.weight_concentration_prior,
                mean_precision_prior=self.mean_precision_prior,
                mean_prior=self.mean_prior,
                degrees_of_freedom_prior=self.degrees_of_freedom_prior,
                covariance_prior=self.covariance_prior,
                random_state=self.random_state,
                warm_start=self.warm_start,
                verbose=self.verbose,
                verbose_interval=self.verbose_interval,
            )
            self.gmms_[c] = mixture.fit(subset_x, subset_y)
            if VERBOSE > 1:
                print(f'Weights of model {c}: {np.sort(np.round(mixture.weights_, 2))}')
        return self


    def predict(self, X):
        check_is_fitted(self, ["gmms_", "classes_"])
        X = check_array(X, estimator=self, dtype=FLOAT_DTYPES)
        return self.classes_[self.predict_proba(X).argmax(axis=1)]


    def predict_proba(self, X):
        X = check_array(X, estimator=self, dtype=FLOAT_DTYPES)
        check_is_fitted(self, ["gmms_", "classes_"])
        res = np.zeros((X.shape[0], self.classes_.shape[0]))
        for idx, c in enumerate(self.classes_):
            res[:, idx] = self.gmms_[c].score_samples(X)
        return softmax(res, axis=1)
    
    
    def get_densities(self, X):
        densities = np.zeros(np.shape(X)[0])
        for c in self.classes_:
            densities += self.gmms_[c].score_samples(X)
        return densities


# Model

In [43]:
def pipeline(X, seed=0):


    X_wo_outliers, y = remove_outliers(X, seed)
    
    predict_proba, _ = update_predictions(X, y, X_wo_outliers, seed)
    
    # Final prediction with full dataset, more n_init and less tol
    if RUN_FINAL_PRED:
        y = np.argmax(predict_proba, axis=1)
        y = pd.Series(y)
        predict_proba = final_prediction(X, y, X_wo_outliers, seed)
    
    return predict_proba
    
    
def update_predictions(X, y, X_wo_outliers=None, seed=0):
    rand_score_prev = 0
#     densities = np.zeros(np.shape(y))
    value_counts = pd.DataFrame(data=np.zeros((len(set(y)), 2)), columns=['count', 'diff'])
    for i in range(N_GMMC_ITERATION):
        if X_wo_outliers is not None:
            X_sample = X_wo_outliers.sample(n=N_SAMPLE, random_state = seed+i)  # None, seed + i
        else:
            X_sample = X.sample(n=N_SAMPLE, random_state = seed+i)  # None, seed + i
        y_sample = y.loc[X_sample.index]
        
        bgmmC = BayesianGMMClassifier(
            n_components = N_CLUSTER, # 1, 6 or N_CLUSTER  !!!!!!!!!!!!!!!!!!!!!!!!!!
            random_state = seed + i,  # None, seed + i
            tol = TOL,
            covariance_type = 'full',
            max_iter = GMM_MAX_ITER,
            n_init=N_INIT,
            init_params=INIT_PARAMS)

        bgmmC.fit(X_sample, y_sample)        
        pred_probs = bgmmC.predict_proba(np.array(X))        
        rand_score = adjusted_rand_score(y, np.argmax(pred_probs, axis=1))
        
        if VERBOSE > 0:
            pct_of_change = 1 - sum(y == np.argmax(pred_probs, axis=1)) / X.shape[0]            
            print('Iter_{} - Pct of change from last iteration: {:.4f}, rand score: {:.4f}'.format(i, pct_of_change, rand_score))
        y = pd.Series(np.argmax(pred_probs, axis=1))
        
        if VERBOSE > 1:
            y_prev = value_counts['count'].copy()
            value_counts['count'] = y.value_counts().sort_index()
            value_counts['diff'] = value_counts['count'] - y_prev
            print(value_counts)
        
        # stop iteration if the rand score decreases and return results of previous iteration
        if rand_score_prev > rand_score and ACTIVATE_EARLY_STOPPING:
            break
        else:
            pred_probs_final = pred_probs
            rand_score_prev = rand_score
#             densities += bgmmC.get_densities(np.array(X))
            densities = bgmmC.get_densities(np.array(X))
        
    return pred_probs_final, densities
            
            
def final_prediction(X, y, X_wo_outliers=None, seed=0):
    """Make the final prediction with the full dataset, low tol, high init"""
    
    if X_wo_outliers is not None:
        X_sample = X_wo_outliers.copy()
    else:
        X_sample = X.copy()
    y_sample = y.loc[X_sample.index]

    bgmmC = BayesianGMMClassifier(
        n_components=N_CLUSTER,
        random_state = seed,
        tol = 0.01,
        covariance_type = 'full',
        max_iter = GMM_MAX_ITER,
        n_init=5,
        init_params=INIT_PARAMS)

    bgmmC.fit(X_sample, y_sample)        
    pred_probs = bgmmC.predict_proba(X)

    
    if VERBOSE > 0:
        pct_of_change = 1 - sum(y == np.argmax(pred_probs, axis=1)) / X.shape[0]
        rand_score = adjusted_rand_score(y, np.argmax(pred_probs, axis=1))
        print('Final pred - Pct of change from last iteration: {:.4f}, rand score: {:.4f}'.format(pct_of_change, rand_score))
        
    return pred_probs

In [44]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


def soft_voting(proba_list):
    # Type here the prediction you may want to keep for the ensembling
    n_models = np.shape(proba_list)[0]  # nb of Models
    
    prediction_to_drop = []
    prediction_to_keep = [i for i in range(n_models) if i not in prediction_to_drop]
    

    values = [i for i in range(N_CLUSTER)]

    ensemble_predict_proba = pd.DataFrame(np.zeros((np.shape(proba_list)[1], N_CLUSTER)), columns = values)

    temp = pd.DataFrame()  # temp dataframe for reshaping and cleaning of predict_proba_list
    # change predict_proba_list from shape (ITERATION, 98000, N_CLUSTER) to (98000, ITERATION * N_CLUSTER)
    for i in range(0, n_models):
        if i in prediction_to_keep:
            temp = pd.concat([temp, pd.DataFrame(proba_list[i])], axis=1)

    #  keep only rows with high probability value
    temp = temp[temp.max(axis=1)>PROBA_THRESHOLD]

    # replace high probabilities by 1, others by 0
    temp[temp > PROBA_THRESHOLD] = 1
    temp[temp < 1] = 0

    # Keep only rows that have more than 90% of the models with high probability
    temp = temp[temp.sum(axis=1) >= n_models * 0.9]

    # Groupy of temp on all columns to find the pattern in the predictions
    temp.columns = [i for i in range(temp.shape[1])]  # rename columns
    mapping = temp.groupby(temp.columns.tolist(),as_index=False).size()

    # Keep the N_CLUSTER biggest group
    mapping = mapping.nlargest(n=N_CLUSTER, columns='size')
    mapping.reset_index(drop=True, inplace=True)

    j = 0
    # Renaming of the clusters
    for i in range(n_models):
        if i in prediction_to_keep:        
            clusters_names = np.argmax(np.array(mapping.iloc[:, j*N_CLUSTER:(j+1)*N_CLUSTER]), axis=1)

            # if one cluster is not renamed, it is affected to the remaining cluster
            remaining_clusters = {}
            if len(set(clusters_names)) < N_CLUSTER:
                remaining_clusters = set(values).difference(set(clusters_names))
                clusters_names[-1] = list(remaining_clusters)[0]
            pred_dict = dict(zip(clusters_names, values))
            
            if VERBOSE > 0:
                print(f'{pred_dict}  clusters clusters not found in pattern: {remaining_clusters}')

            pred_temp = pd.DataFrame(proba_list[i]).rename(columns = pred_dict)
            pred_temp = pred_temp.reindex(sorted(pred_temp.columns), axis=1)
            ensemble_predict_proba += pred_temp # Soft voting by probabiliy addition        
            j += 1

    ensemble_predict_proba = ensemble_predict_proba / len(prediction_to_keep)
    index_trusted = ensemble_predict_proba.max(axis=1) > TRUST_THRESHOLD
    ensemble_predict = np.argmax(np.array(ensemble_predict_proba), axis=1)
    if VERBOSE > 0:
        print()
        print(pd.DataFrame(ensemble_predict).value_counts())
    # print(temp.groupby(temp.columns.tolist(),as_index=False).size().sort_values(by='size', ascending=False).head(20))
    return ensemble_predict, index_trusted

In [45]:
def remove_outliers(X, seed=0):
    if VERBOSE > 0:
        print('====== Remove Outliers ======')
    if OUTLIERS_THRESHOLD > 0:
        X_sample = X.sample(n=N_SAMPLE, random_state = seed)
        gmm = BayesianGaussianMixture(
            n_components=N_CLUSTER, 
            covariance_type='full',
            init_params=INIT_PARAMS,  # 'kmeans', 'k-means++', 'random', 'random_from_data'}
            random_state = seed,
            tol = TOL,
            max_iter=GMM_MAX_ITER, 
            verbose=0, 
            verbose_interval=50, 
            n_init = N_INIT
        )
        gmm.fit(X_sample)
        y = pd.Series(gmm.predict(X))    
    
        _, densities = update_predictions(X, y, seed=seed)
        density_threshold = np.percentile(densities, OUTLIERS_THRESHOLD*100)
        X_wo_outliers = X[densities > density_threshold]
    return X_wo_outliers, y

In [46]:
# For test: remove_outliers
# %%time
# N_CLUSTER = 7
# N_GMMC_ITERATION = 30
# N_MODELS = 100
# GMM_MAX_ITER = 300
# N_SAMPLE = 20000
# ACTIVATE_EARLY_STOPPING = True
# TOL = 0.01
# N_INIT = 3
# PROBA_THRESHOLD = 0.99  # Rows with probability lower than this threshold are dropped
# RUN_FINAL_PRED = True  # Make the final prediction with the full dataset, low tol, high init
# EARLY_STOPPING_ENSEMBLE_RAND = 0.99
# TRUST_THRESHOLD = 0.7
# OUTLIERS_THRESHOLD = 0.01
# VERBOSE = 2
# seed = 0

# df1 = df_scaled[:20000]
# df2 = remove_outliers(df1)

In [47]:
%%time
N_CLUSTER = 7
N_GMMC_ITERATION = 20
N_MODELS = 30
GMM_MAX_ITER = 300
N_SAMPLE = 50000
ACTIVATE_EARLY_STOPPING = True
TOL = 0.01
N_INIT = 3
PROBA_THRESHOLD = 0.99  # Rows with probability lower than this threshold are dropped
RUN_FINAL_PRED = True  # Make the final prediction with the full dataset, low tol, high init
EARLY_STOPPING_ENSEMBLE_RAND = 0.998
TRUST_THRESHOLD = 0.7
OUTLIERS_THRESHOLD = 0.01
VERBOSE = 0
INIT_PARAMS='k-means++'  # 'kmeans', 'k-means++', 'random', 'random_from_data'}
seed = 0

X = df_scaled
ensemble_predict_prev = -np.ones(X.shape[0])

proba_list = []
for _ in trange(N_MODELS):
    proba = pipeline(X, seed=seed)
    proba_list.append(proba)
    print('====== Ensemble ======')
    ensemble_predict, index_trusted = soft_voting(proba_list)
    pct_of_change = 1 - sum(ensemble_predict == ensemble_predict_prev) / X.shape[0]
    rand_score = adjusted_rand_score(ensemble_predict, ensemble_predict_prev)    
    print('Pct of ensemble prediction change from last iteration: {:.4f}, rand score: {:.4f}'.format(pct_of_change, rand_score))
    ensemble_predict_prev = ensemble_predict
    seed += N_GMMC_ITERATION
    time.sleep(1)  # wait 1 second for correct printing of trange
    if rand_score > EARLY_STOPPING_ENSEMBLE_RAND:
        break
    
print(pd.DataFrame(ensemble_predict).value_counts())

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]C:\Users\hzf04d\Anaconda3\envs\MainEnv\lib\site-packages\sklearn\mixture\_base.py:286: ConvergenceWarning: Initialization 3 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


====== Ensemble ======
Pct of ensemble prediction change from last iteration: 1.0000, rand score: 0.0000


  3%|██▋                                                                             | 1/30 [05:48<2:48:30, 348.63s/it]

====== Ensemble ======
Pct of ensemble prediction change from last iteration: 0.8113, rand score: 0.8220


  7%|█████▎                                                                          | 2/30 [11:09<2:35:04, 332.29s/it]

...

====== Ensemble ======
Pct of ensemble prediction change from last iteration: 0.0015, rand score: 0.9966


 37%|████████████████████████████▏                                                | 11/30 [1:08:27<2:00:20, 380.04s/it]

====== Ensemble ======
Pct of ensemble prediction change from last iteration: 0.0008, rand score: 0.9980


 37%|████████████████████████████▏                                                | 11/30 [1:13:47<2:07:28, 402.54s/it]

3    16375
2    16223
5    14834
1    13844
0    13162
6    12223
4    11339
dtype: int64
Wall time: 1h 13min 47s


# Submission

In [48]:
# sub = pd.read_csv('../input/tabular-playground-series-jul-2022/sample_submission.csv')
sub = pd.read_csv('data\sample_submission.csv')
sub['Predicted'] = ensemble_predict
sub.to_csv('submission.csv', index=False)